# **Data Processing y Series**




Jhon Esteban Arredondo Parra - 1035443205

Gerson Gabriel Erira - 1085931900

## **Objetivo**
Definir las bases de datos para modelos de Machine learning y Deep Learning que predigan las ventas de productos masivos en 5 departamentos de Colombia, con el fin de buscar el mejor modelo que aporte a la identificación de posibles preferencias de compra, tendencias de consumo y con esto proyectar un comportamiento por subcategoría de producto.

In [ ]:
#downgrade tensorflow tensorflow==2.15
# !pip install tensorflow==2.15

In [ ]:
#Cargar datos de Github
import io
import requests


#Manipulacion de datos
import pandas as pd
import numpy as np
import os
from collections import Counter


#Separar los datos entrenamiento y prueba
from sklearn.model_selection import train_test_split


# Gráficos
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import style
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px


##evitar warnings
import warnings
warnings.filterwarnings('ignore')

# 1. Cargar todas las Tablas

El dataset inicial es llamado compras, contiene 16383 registros de venta de productos de consumo (supermercado), la fecha, ID de municipio, CANTIDAD, UNIDAD de medición del producto y el costo TOTAL. Se busca adicionar variables de tiempo, socioeconomicas, y la categoria de productos para la estimación de cantidades vendidas

In [ ]:
rescompras = requests.get('https://github.com/Gabriel93UdeA/AprendizajeAutomatico/raw/main/ComprasDda.xlsx')
rescompras.raise_for_status()

compras = pd.read_excel(io.BytesIO(rescompras.content), sheet_name="Compras2022", engine='openpyxl')
# compras = pd.read_excel("ComprasDda.xlsx", sheet_name="Compras2022", engine='openpyxl')
compras = compras[["PRODUCTO", "FECHA", "IDUbicacion", "CANTIDAD", "UNIDAD", "TOTAL"]]
compras.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16383 entries, 0 to 16382
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   PRODUCTO     16383 non-null  object        
 1   FECHA        16383 non-null  datetime64[ns]
 2   IDUbicacion  16383 non-null  int64         
 3   CANTIDAD     16383 non-null  float64       
 4   UNIDAD       16383 non-null  object        
 5   TOTAL        16383 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 768.1+ KB


In [ ]:
resubicaciones = requests.get('https://github.com/Gabriel93UdeA/AprendizajeAutomatico/raw/main/Ubicaciones.xlsx')
resubicaciones.raise_for_status()
ubicaciones = pd.read_excel(io.BytesIO(resubicaciones.content), sheet_name="Ubicacion")
# ubicaciones = pd.read_excel("Ubicaciones.xlsx", sheet_name="Ubicacion", engine='openpyxl')
ubicaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1725 entries, 0 to 1724
Data columns (total 11 columns):
 #   Column                                                                           Non-Null Count  Dtype 
---  ------                                                                           --------------  ----- 
 0   Ubicación                                                                        1725 non-null   int64 
 1   IDDto                                                                            1725 non-null   int64 
 2   IDMpio                                                                           1725 non-null   int64 
 3   Dpto                                                                             1725 non-null   object
 4   Municipio                                                                        1725 non-null   object
 5   Subregión (SGR)                                                                  1725 non-null   object
 6   Categoría de rur

In [ ]:
# este metodo lee todas las hojas del archivo de excel en una sola tabla
resvariables = requests.get('https://github.com/Gabriel93UdeA/AprendizajeAutomatico/raw/main/Variables.xlsx')
resvariables.raise_for_status()

# Leer todas las hojas de una vez
xls = pd.ExcelFile(io.BytesIO(resvariables.content))
dfs = pd.read_excel(xls, sheet_name=None, usecols=lambda col: col not in ["Dato Cualitativo", "Mes", "Fuente", "Unidad de Medida"])
# dfs = pd.read_excel("Variables.xlsx", sheet_name=None, usecols=lambda col: col not in ["Dato Cualitativo", "Mes", "Fuente", "Unidad de Medida"], engine='openpyxl')

# Concatenar y filtrar los DataFrames
variables_socio = pd.concat([
    df[(df["Año"] <= 2022) & (df["Año"] >= 2017) & (df["Entidad"] != "Colombia")]
    for df in dfs.values()
])

variables_socio.reset_index(drop=True, inplace=True)

print("Todas las hojas leídas y fusionadas.")

##se necesita estandarizar las tablas
variables_socio.loc[(variables_socio["Dato Numérico"].astype(str).str.contains(",")), "Dato Numérico"] = variables_socio.loc[(variables_socio["Dato Numérico"].astype(str).str.contains(",")), "Dato Numérico"].astype(str).str.replace(",", ".")

# Función para limpiar y convertir los valores
def clean_and_convert(value):
    if isinstance(value, str) and '.' in value:
        parts = value.split('.')
        if len(parts) == 3:
            cleaned_value = parts[0] + parts[1] + '.' + parts[2]
            return float(cleaned_value)
    return value

# Aplicar la función a la columna
variables_socio["Dato Numérico"] = variables_socio["Dato Numérico"].apply(clean_and_convert)
variables_socio["Dato Numérico"] = variables_socio["Dato Numérico"].astype(float)

##como tenemos los registros de varios años entre 2017 y 2022, deseamos tener solo un registro por municipio, por lo que sacamomos el valor
#promedio para cada municipio en ese rango de años
variables_socio = variables_socio.pivot_table(index=["Código Departamento", "Código Entidad"],
                                              columns="Indicador", values="Dato Numérico", aggfunc="mean").reset_index()

variables_socio.info()

Todas las hojas leídas y fusionadas.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1133 entries, 0 to 1132
Data columns (total 51 columns):
 #   Column                                                                                                                           Non-Null Count  Dtype  
---  ------                                                                                                                           --------------  -----  
 0   Código Departamento                                                                                                              1133 non-null   int64  
 1   Código Entidad                                                                                                                   1133 non-null   int64  
 2   Cuarta medida correctiva al Código Nacional de Policía más impuesta en  la entidad territorial                                   1111 non-null   float64
 3   Fallecidos por siniestros viales                             

In [ ]:
##categorias del modelo de clasificacion ajustado
categorias = pd.read_excel('/content/ComprasCategorizadas_SVC.xlsx')

##limpiar y seleccionar solo columnas necesarias
categorias = categorias[["PRODUCTO", "CATEGORIA"]]
categorias.columns = ["PRODUCTO", "subcategory"]
categorias["subcategory"] = categorias["subcategory"].map(lambda x: x.replace("'", "").replace("[", "").replace("]", "").strip())

categorias.info()

In [ ]:
resfecha = requests.get('https://github.com/Gabriel93UdeA/AprendizajeAutomatico/raw/main/Fechas.xlsx')
resfecha.raise_for_status()
fecha_df = pd.read_excel(io.BytesIO(resfecha.content))

# fecha_df = pd.read_excel("Fechas.xlsx", engine='openpyxl')
fecha_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Fecha     365 non-null    datetime64[ns]
 1   Dia       365 non-null    object        
 2   Tipo Dia  365 non-null    object        
dtypes: datetime64[ns](1), object(2)
memory usage: 8.7+ KB


# 2. Merge Entre Tablas
Llegar al dataset basico

In [ ]:
##merge de categorias de productos
df = compras.merge(categorias[["PRODUCTO", "subcategory"]], left_on="PRODUCTO", right_on="PRODUCTO", how="left")
df.sample()

,PRODUCTO,FECHA,IDUbicacion,CANTIDAD,UNIDAD,TOTAL,subcategory
236380,MANI JOSSEB *210G,2022-04-05,1619,6.0,UND,20400,Pasabocas


In [ ]:
df.shape

(301101, 7)

El merge anterior genera registros duplicados. Se eliminan a continuación para evitar errores futuros

In [ ]:
df.drop_duplicates(inplace=True)
df.shape

(16383, 7)

In [ ]:
##merge de fechas y tipo de dias
df = df.merge(fecha_df, left_on="FECHA", right_on="Fecha", how="left")
df.sample()

,PRODUCTO,FECHA,IDUbicacion,CANTIDAD,UNIDAD,TOTAL,subcategory,Fecha,Dia,Tipo Dia
14063,CHOKIS CHISPAS CLASICA *37G,2022-01-29,849,6.0,UND,4446,Pasabocas,2022-01-29,Sabado,Fin de Semana


In [ ]:
df.shape

(16383, 10)

In [ ]:
##merge de municipio y regiones
df = df.merge(ubicaciones[['IDDto', 'IDMpio', 'Ubicación', 'Dpto', 'Municipio', 'Subregión (SGR)']], left_on="IDUbicacion", right_on="Ubicación", how="left")

#identificar subregion
df['Subregión (SGR)'] = df['Dpto'] + ' - ' + df['Subregión (SGR)'] #para diferenciar las subregiones de mismo nombre y dpto distinto
df['Subregión (SGR)'].value_counts()

df.sample()

,PRODUCTO,FECHA,IDUbicacion,CANTIDAD,UNIDAD,TOTAL,subcategory,Fecha,Dia,Tipo Dia,IDDto,IDMpio,Ubicación,Dpto,Municipio,Subregión (SGR)
16373,PROTECTORES CAREFREE*60,2022-04-19,1556,12.0,UND,84900,Cuidado Personal,2022-04-19,Martes,Laboral,76,76520,1556,VALLE DEL CAUCA,PALMIRA,VALLE DEL CAUCA - Sur


In [ ]:
df.shape

(16383, 16)

In [ ]:
##merge variables socioeconomicas de la region y municipio
df = df.merge(variables_socio,
              left_on=['IDDto', 'IDMpio'], right_on=['Código Departamento', 'Código Entidad'], how="left")
df.sample()

,PRODUCTO,FECHA,IDUbicacion,CANTIDAD,UNIDAD,TOTAL,subcategory,Fecha,Dia,Tipo Dia,...,Tasa de violencia contra niños y niñas de 6 a 11 años,Tasa de violencia contra niños y niñas de primera infancia,Tasa de violencia de pareja a mujeres por cada 100 mil habitantes,Tasa de violencia interpersonal por cada 100.000 habitantes,Tasa de violencia intrafamiliar en adolescentes (12 a 17 años),Tasa de violencia intrafamiliar en niños y niñas de 0 a 5 años,Tasa de violencia intrafamiliar en niños y niñas de 6 a 11 años,Tasa de violencia intrafamiliar por cada 100.000 habitantes,Tercera medida correctiva al Código Nacional de Policía más impuesta en la entidad territorial,Índice de Interrelación de Problemáticas
744,TENA MUJER SUPER *10UND,2022-02-02,1139,3.0,UND,43746,Belleza,2022-02-02,Miercoles,Laboral,...,460.8625,259.0375,110.69,263.493333,159.3375,83.89,131.453333,127.0325,5.74,40.99


In [ ]:
##dataset resultante
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16383 entries, 0 to 16382
Data columns (total 67 columns):
 #   Column                                                                                                                           Non-Null Count  Dtype         
---  ------                                                                                                                           --------------  -----         
 0   PRODUCTO                                                                                                                         16383 non-null  object        
 1   FECHA                                                                                                                            16383 non-null  datetime64[ns]
 2   IDUbicacion                                                                                                                      16383 non-null  int64         
 3   CANTIDAD                                                     

In [ ]:
df.shape

(16383, 67)

# 3. Analisis Previo de las Series (Datasets originales)

A continuación, realizamos la exploración de datos de forma inicial, centrado en las series de tiempo y en los comportamientos sobre las principales regiones y categorias. Posteriormente en la sección 4 se hará una depuración de información, en la 5 un EDA detallado incluyendo relaciones entre variables, en 6 una limpieza y agrupación de acuerdo al nivel de detalle requerido, en 7 y 8 se preparan los datos y entrenan los modelos.


In [ ]:
## serie de tiempo agregada de todas las subregiones y categorias
df_total = df.groupby(['FECHA'])['CANTIDAD'].sum().reset_index()

##plotly line series
fig = px.line(df_total, x="FECHA", y="CANTIDAD", title=f'Cantidades vendidas diariamente en los 5 departamentos')
fig.show()

## Serie Total por Semana y quincena



In [ ]:
# Agrupación semanal (por suma de cantidades) ----------------------------------
df_semanal = df_total.resample('W', on='FECHA').sum().reset_index()


# Crear una columna de quincena (1-15, 16-fin de mes) --------------------------------------------
df_total['quincena'] = df_total['FECHA'].dt.day.apply(lambda x: 1 if x <= 15 else 2)

# Agrupar por mes y quincena
df_total['periodo_quincenal'] = df_total['FECHA'].dt.to_period('M').astype(str) + "-Q" + df_total['quincena'].astype(str)

# Agrupar y sumar por quincena
df_quincenal = df_total.groupby('periodo_quincenal', as_index=False)['CANTIDAD'].sum()

In [ ]:
fig_semanal = px.line(df_semanal, x='FECHA', y='CANTIDAD',
                      title='Serie de Tiempo Semanal',
                      labels={'FECHA': 'Semana', 'CANTIDAD': 'Cantidad'},
                      color_discrete_sequence=px.colors.cyclical.IceFire)
fig_semanal.update_layout(template='gridon', title_font_size=16)
fig_semanal.show()

In [ ]:
fig_quincenal = px.line(df_quincenal, x='periodo_quincenal', y='CANTIDAD',
                       title='Serie de Tiempo Quincenal',
                       labels={'periodo_quincenal': 'Quincena', 'CANTIDAD': 'Cantidad'},
                       color_discrete_sequence=px.colors.cyclical.IceFire)
fig_quincenal.update_traces(marker=dict(line=dict(color='black', width=1)))
fig_quincenal.update_layout(title_font_size=18, template='gridon')
fig_quincenal.show()

## Principales categorias

In [ ]:
df['subcategory'].value_counts().head(4).index

Index(['Frutas y Verduras', 'Pasabocas', 'Despensa', 'Dulces y postres'], dtype='object', name='subcategory')

In [ ]:
top_subcategorias = df['subcategory'].value_counts().head(4).index
df_sub = df[df['subcategory'].isin(top_subcategorias)]
# df_sub = df_sub.groupby(['subcategory', 'FECHA'])['CANTIDAD'].sum().reset_index()
df_sub['week'] = df_sub['FECHA'].dt.isocalendar().week
df_semanal = df_sub.groupby(['subcategory', 'week'])['CANTIDAD'].sum().reset_index()
df_semanal.sample()

,subcategory,week,CANTIDAD
55,Dulces y postres,4,263.0


In [ ]:
fig = px.line(df_semanal.set_index('week'), facet_col="subcategory", facet_col_wrap=2,
              color='subcategory',
              color_discrete_sequence=px.colors.qualitative.Bold_r)
fig.update_layout(template='ggplot2', title_font_size=16)
fig.show()

In [ ]:
# Crear una columna de quincena (1-15, 16-fin de mes) --------------------------------------------
df_sub['quincena'] = df_sub['FECHA'].dt.day.apply(lambda x: 1 if x <= 15 else 2)

# Agrupar por mes y quincena
df_sub['periodo_quincenal'] = df_sub['FECHA'].dt.to_period('M').astype(str) + "-Q" + df_sub['quincena'].astype(str)

# Agrupar y sumar por quincena
df_quincenal = df_sub.groupby(['periodo_quincenal', 'subcategory'], as_index=False)['CANTIDAD'].sum()
df_quincenal.sample()

,periodo_quincenal,subcategory,CANTIDAD
35,2022-05-Q1,Pasabocas,1024.0


In [ ]:
fig = px.line(df_quincenal, x='periodo_quincenal', y='CANTIDAD',
                       title='Serie de Tiempo Quincenal', facet_col="subcategory", facet_col_wrap=2,
              color_discrete_sequence=px.colors.qualitative.Bold,
              color='subcategory')
fig.update_layout(template='ggplot2', title_font_size=16)
fig.show()

## Series por subcategoria y Departamento

In [ ]:
df['Dpto'].value_counts().head(5).index

Index(['ANTIOQUIA', 'VALLE DEL CAUCA', 'CALDAS', 'RISARALDA', 'QUINDÍO'], dtype='object', name='Dpto')

In [ ]:
top_subcategorias = df['subcategory'].value_counts().head(4).index
df_sub = df[df['subcategory'].isin(top_subcategorias)]
df_sub = df_sub[df_sub['Dpto'].isin(df['Dpto'].value_counts().head(5).index)]
df_sub['week'] = df_sub['FECHA'].dt.isocalendar().week
df_semanal = df_sub.groupby(['subcategory', 'Dpto', 'week'])['CANTIDAD'].sum().reset_index()
df_semanal.sample()

,subcategory,Dpto,week,CANTIDAD
10,Despensa,ANTIOQUIA,11,149.0


In [ ]:
fig = px.line(
    df_semanal,
    x='week',
    y='CANTIDAD',
    color='Dpto',  # Colores por subregión
    facet_col='subcategory',  # Subcategorías en columnas
    facet_row='Dpto',  # Subregiones en filas
    color_discrete_sequence=px.colors.qualitative.Bold,  # Paleta de colores
)

# Personalizar diseño
fig.update_layout(
    template='ggplot2',
    title="Serie Semanal por Subcategoría y Subregión",
    title_font_size=16
)

fig.show()

In [ ]:
# Obtener las subcategorías principales
top_subcategorias = df['subcategory'].value_counts().head(5).index

# Filtrar el DataFrame por subcategorías y subregiones principales
df_sub = df[df['subcategory'].isin(top_subcategorias)]
df_sub = df_sub[df_sub['Dpto'].isin(df['Dpto'].value_counts().head(5).index)]

# Calcular la quincena (1 o 2) en función de la fecha
df_sub['quincena'] = df_sub['FECHA'].dt.day.apply(lambda x: 1 if x <= 15 else 2)

# Agrupar por mes y quincena
df_sub['periodo_quincenal'] = df_sub['FECHA'].dt.to_period('M').astype(str) + "-Q" + df_sub['quincena'].astype(str)


# Agrupar los datos por subcategoría, subregión y quincena
df_quincenal = df_sub.groupby(['subcategory', 'Dpto', 'periodo_quincenal'])['CANTIDAD'].sum().reset_index()

# Mostrar una muestra del resultado
df_quincenal.sample()

,subcategory,Dpto,periodo_quincenal,CANTIDAD
85,Cuidado Personal,VALLE DEL CAUCA,2022-02-Q2,354.5


In [ ]:
px.line(df_quincenal[(df_quincenal['Dpto']=='VALLE DEL CAUCA') & (df_quincenal['subcategory']=='Despensa')],
        x='periodo_quincenal', y='CANTIDAD')

In [ ]:
fig = px.line(
    df_quincenal.sort_values(by="periodo_quincenal", ascending=True),
    x='periodo_quincenal',
    y='CANTIDAD',
    color='Dpto',  # Colores por subregión
    facet_col='subcategory',  # Subcategorías en columnas
    facet_row='Dpto',  # Subregiones en filas
    color_discrete_sequence=px.colors.qualitative.Bold,  # Paleta de colores
)

# Personalizar diseño
fig.update_layout(
    template='ggplot2',
    title="Serie Quincenal por Subcategoría y Subregión",
    title_font_size=16
)

fig.show()

EL GRAFICO ANTERIOR SE DAÑA EL ORDEN PORQUE PARA LA SUBREGION URABA Y CATEGORIA DESPENSA NO HAY DATOS EN LA QUINCENA 07-Q1!

In [ ]:
##llenar con cero las quincenas que no existen en la combinacion region-subcategoria
#Crear todas las combinaciones posibles
regiones = df_quincenal["Dpto"].unique()
subcategorias = df_quincenal["subcategory"].unique()
quincenas = df_quincenal['periodo_quincenal'].unique()

combinaciones = pd.MultiIndex.from_product([regiones, subcategorias, quincenas], names=["Dpto", "subcategory", "periodo_quincenal"])
df_completo = pd.DataFrame(index=combinaciones).reset_index()

#Hacer un merge con los datos originales
df_quincenal_nuevo = df_completo.merge(df_quincenal, on=["Dpto", "subcategory", "periodo_quincenal"], how="left")

#Llenar los valores faltantes con 0
df_quincenal_nuevo["CANTIDAD"] = df_quincenal_nuevo["CANTIDAD"].fillna(0)

df_quincenal_nuevo


,Dpto,subcategory,periodo_quincenal,CANTIDAD
0,ANTIOQUIA,Cuidado Personal,2022-01-Q1,226.0
1,ANTIOQUIA,Cuidado Personal,2022-01-Q2,439.0
2,ANTIOQUIA,Cuidado Personal,2022-02-Q1,564.0
3,ANTIOQUIA,Cuidado Personal,2022-02-Q2,326.0
4,ANTIOQUIA,Cuidado Personal,2022-03-Q1,557.0
...,...,...,...,...
595,VALLE DEL CAUCA,Pasabocas,2022-10-Q2,147.0
596,VALLE DEL CAUCA,Pasabocas,2022-11-Q1,101.0
597,VALLE DEL CAUCA,Pasabocas,2022-11-Q2,61.0
598,VALLE DEL CAUCA,Pasabocas,2022-12-Q1,122.0


In [ ]:
fig = px.line(
    df_quincenal_nuevo.sort_values(by="periodo_quincenal", ascending=True),
    x='periodo_quincenal',
    y='CANTIDAD',
    color='Dpto',  # Colores por subregión
    facet_col='subcategory',  # Subcategorías en columnas
    facet_row='Dpto',  # Subregiones en filas
    color_discrete_sequence=px.colors.qualitative.Bold,  # Paleta de colores
)

# Personalizar diseño
fig.update_layout(
    template='ggplot2',
    title="Serie Quincenal por Subcategoría y Subregión",
    title_font_size=16
)

fig.show()

# Definir tiempo

In [ ]:
# Semanal
df['week'] = df['FECHA'].dt.isocalendar().week
df.sample()

,PRODUCTO,FECHA,IDUbicacion,CANTIDAD,UNIDAD,TOTAL,subcategory,Fecha,Dia,Tipo Dia,...,Tasa de violencia contra niños y niñas de primera infancia,Tasa de violencia de pareja a mujeres por cada 100 mil habitantes,Tasa de violencia interpersonal por cada 100.000 habitantes,Tasa de violencia intrafamiliar en adolescentes (12 a 17 años),Tasa de violencia intrafamiliar en niños y niñas de 0 a 5 años,Tasa de violencia intrafamiliar en niños y niñas de 6 a 11 años,Tasa de violencia intrafamiliar por cada 100.000 habitantes,Tercera medida correctiva al Código Nacional de Policía más impuesta en la entidad territorial,Índice de Interrelación de Problemáticas,week
8674,BROCOLI,2022-12-31,1364,6.0,UND,16002,Frutas y Verduras,2022-12-31,Sabado,Fin de Semana,...,153.185,297.88,293.583333,152.7575,64.9575,179.283333,220.37,1.326667,13.95,52


In [ ]:
df.columns

Index(['PRODUCTO', 'FECHA', 'IDUbicacion', 'CANTIDAD', 'UNIDAD', 'TOTAL',
       'subcategory', 'Fecha', 'Dia', 'Tipo Dia', 'IDDto', 'IDMpio',
       'Ubicación', 'Dpto', 'Municipio', 'Subregión (SGR)',
       'Código Departamento', 'Código Entidad',
       'Cuarta medida correctiva al Código Nacional de Policía más impuesta en  la entidad territorial',
       'Fallecidos por siniestros viales', 'Población total de hombres',
       'Población total de mujeres', 'Porcentaje población de hombres',
       'Porcentaje población de mujeres',
       'Primera medida correctiva al Código Nacional de Policía más impuesta en  la entidad territorial',
       'Proporción en que son impuestas otras medidas correctivas al Código Nacional de Policía más impuesta en  la entidad territorial',
       'Puntaje SISBEN: Promedio - Cabecera',
       'Quinta medida correctiva al Código Nacional de Policía más impuesta en  la entidad territorial',
       'Segunda medida correctiva al Código Nacional de Policí

In [ ]:
df.shape

(16383, 68)

# 4. Analisis y Limpieza Previa de datos
En esta sección se realiza una revisión superficial al dataset, con el fin de detectar y corregir falencias propias de la estructura de datos.

Adicionalmente, se realiza la agregación de las variables por Fecha, Municipio, Subregion, Departamento y Categoria de Producto. Las cantidades serán sumadas y los indicadores socioeconomicos se promediaran por municipio.

☑️ Realizado, decision por tomar
✅ Realizado y definitivo
➖ Falta realizar
✖️ Codigo comentado, para no intervenir el dataset aun

## Unidades ☑️
Los productos originalmente tienen distitnas unidades de medida. A continuación estandarizamos las unidades por cantidad y peso, según apliquen

In [ ]:
df['UNIDAD'].value_counts()

,count
UNIDAD,
UND,13238
KILO,2014
PQT,593
BDJ,185
CJ,77
OFT,61
@,48
BTO,39
LB,31


In [ ]:
df.loc[df["UNIDAD"]=='LB', ].sample()

,PRODUCTO,FECHA,IDUbicacion,CANTIDAD,UNIDAD,TOTAL,subcategory,Fecha,Dia,Tipo Dia,...,Tasa de violencia contra niños y niñas de primera infancia,Tasa de violencia de pareja a mujeres por cada 100 mil habitantes,Tasa de violencia interpersonal por cada 100.000 habitantes,Tasa de violencia intrafamiliar en adolescentes (12 a 17 años),Tasa de violencia intrafamiliar en niños y niñas de 0 a 5 años,Tasa de violencia intrafamiliar en niños y niñas de 6 a 11 años,Tasa de violencia intrafamiliar por cada 100.000 habitantes,Tercera medida correctiva al Código Nacional de Policía más impuesta en la entidad territorial,Índice de Interrelación de Problemáticas,week
13194,MANI OLIMANI *200G SURTIDA,2022-04-26,315,16.0,LB,54400,Pasabocas,2022-04-26,Martes,Laboral,...,20.2925,0.0,98.596667,99.4,110.74,198.22,98.73,2.713333,43.82,17


In [ ]:
##corregir errores dataset
df.loc[df["UNIDAD"]==2265, "UNIDAD"] = "UND"
df.loc[df["UNIDAD"]==2365, "UNIDAD"] = "UND"

## estandarización UNIDAD por cantidad
df['UNIDAD'] = df['UNIDAD'].str.upper()
df['UNIDAD'] = df['UNIDAD'].str.strip()

df.loc[df["UNIDAD"]=="PQT", "UNIDAD"] = "UND" #paquete
df.loc[df["UNIDAD"]=='BDJ', "UNIDAD"] = "UND" #bandeja
df.loc[df["UNIDAD"]=='TARR', "UNIDAD"] = "UND" #tarro
df.loc[df["UNIDAD"]=='CJ', "UNIDAD"] = "UND" #caja
df.loc[df["UNIDAD"]=='PAR', "UNIDAD"] = "UND" #par
df.loc[df["UNIDAD"]=='DOC', "UNIDAD"] = "UND" #docena
df.loc[df["UNIDAD"]=='ATAD', "UNIDAD"] = "UND" #atado
df.loc[df["UNIDAD"]=='DCN', "UNIDAD"] = "UND" #decena
df.loc[df["UNIDAD"]=='CTN', "UNIDAD"] = "UND" #carton
df.loc[df["UNIDAD"]=='PCA', "UNIDAD"] = "UND" #paca
df.loc[df["UNIDAD"]=='OFT', "UNIDAD"] = "UND" #oferta
df.loc[df["UNIDAD"]=='LB', "UNIDAD"] = "UND" #unidad


##estandarizacion UNIDAD por peso
df.loc[df["UNIDAD"]=='BTO', "CANTIDAD"] = df.loc[df["UNIDAD"]=='BTO', "CANTIDAD"]*40 #peso promedio de un bulto
df.loc[df["UNIDAD"]=='BTO', "UNIDAD"] = "KILO"

# df.loc[df["UNIDAD"]=='LB', "CANTIDAD"] = df.loc[df["UNIDAD"]=='LB', "CANTIDAD"]*0.5 #libra es medio kilo
# df.loc[df["UNIDAD"]=='LB', "UNIDAD"] = "KILO"

df.loc[df["UNIDAD"]=='@', "CANTIDAD"] = df.loc[df["UNIDAD"]=='@', "CANTIDAD"]*12.5 #arroba es 12.5 kilos
df.loc[df["UNIDAD"]=='@', "UNIDAD"] = "KILO"

In [ ]:
df[['subcategory', 'UNIDAD']].value_counts()

,,count
subcategory,UNIDAD,
Frutas y Verduras,UND,3216
Pasabocas,UND,2419
Frutas y Verduras,KILO,1822
Despensa,UND,1545
Dulces y postres,UND,1263
Cuidado Personal,UND,1012
"Lácteos, huevos y refrigerados",UND,883
Limpieza de cocina,UND,654
Utiles Escolares,UND,613


In [ ]:
##verificamos si los productos tienen solo una unidad
val = df[['PRODUCTO', 'UNIDAD']].value_counts().reset_index()
print(val[val.duplicated(subset=['PRODUCTO'], keep=False)])


#corregimos para YUCA y MANZANA ROJA que la UNIDAD sea KILO
df.loc[df["PRODUCTO"]=='YUCA', "UNIDAD"] = "KILO"
df.loc[df["PRODUCTO"]=='MANZANA ROJA', "UNIDAD"] = "KILO"

          PRODUCTO UNIDAD  count
21            YUCA   KILO     47
35    MANZANA ROJA    UND     42
2182          YUCA    UND      1
2238  MANZANA ROJA   KILO      1


In [ ]:
df[(df['subcategory']=='Cuidado Personal') & (df['UNIDAD']=='KILO')]

,PRODUCTO,FECHA,IDUbicacion,CANTIDAD,UNIDAD,TOTAL,subcategory,Fecha,Dia,Tipo Dia,...,Tasa de violencia contra niños y niñas de primera infancia,Tasa de violencia de pareja a mujeres por cada 100 mil habitantes,Tasa de violencia interpersonal por cada 100.000 habitantes,Tasa de violencia intrafamiliar en adolescentes (12 a 17 años),Tasa de violencia intrafamiliar en niños y niñas de 0 a 5 años,Tasa de violencia intrafamiliar en niños y niñas de 6 a 11 años,Tasa de violencia intrafamiliar por cada 100.000 habitantes,Tercera medida correctiva al Código Nacional de Policía más impuesta en la entidad territorial,Índice de Interrelación de Problemáticas,week
3265,ITALCAN PLUS,2022-05-21,264,200.0,KILO,603711,Cuidado Personal,2022-05-21,Sabado,Fin de Semana,...,78.8850,87.01,157.063333,77.555000,19.220000,42.507500,135.7175,1.080000,0.00,20
3266,ITALCAN PLUS *25KG,2022-05-21,125,320.0,KILO,603711,Cuidado Personal,2022-05-21,Sabado,Fin de Semana,...,167.2700,112.31,202.010000,37.486667,66.553333,92.716667,81.9025,3.195000,38.01,20
11372,ITALCAN PLUS,2022-02-21,1548,0.5,KILO,1366,Cuidado Personal,2022-02-21,Lunes,Laboral,...,134.3125,49.10,119.706667,79.940000,161.640000,124.005000,46.6900,2.626667,41.90,8
12871,ITALCAN PLUS,2022-04-14,1194,25.0,KILO,73340,Cuidado Personal,2022-04-14,Jueves,Festivo,...,103.1000,160.38,248.880000,49.780000,16.542500,27.445000,184.1275,2.406667,10.35,15


In [ ]:
##Solo consideraremos inicialmente los productos que son en UNIDAD
df = df[compras["UNIDAD"]=="UND"]

## Tiendas 🚨

In [ ]:
#calcular el mes a partir de la fecha
df['mes'] = df['FECHA'].dt.month
print("Registros 'mes':")
print(df['mes'].unique())

df['dia_num'] = df['FECHA'].dt.day

##numero de tiendas distintas por mes y region
val = df.groupby(['mes', 'Dpto'])['IDUbicacion'].nunique().reset_index().sort_values(by="IDUbicacion", ascending=False)
##pivot para ver el mes en multiples columnas, index subregion
val.pivot(index="Dpto", columns="mes", values="IDUbicacion")

Registros 'mes':
[ 1  2  3  4  5  6  7  8  9 11 12 10]


mes,1,2,3,4,5,6,7,8,9,10,11,12
Dpto,,,,,,,,,,,,
ANTIOQUIA,400,399,441,407,346,382,327,376,342,174,161,219
CALDAS,116,121,115,98,108,99,81,111,108,43,47,67
QUINDÍO,48,46,50,47,39,49,35,55,49,19,19,31
RISARALDA,82,83,95,80,71,78,69,85,77,36,27,48
VALLE DEL CAUCA,297,293,303,320,297,296,251,289,258,130,110,179


In [ ]:
val = df.groupby(['mes'])['IDUbicacion'].nunique().reset_index().sort_values(by="mes", ascending=True)
val

,mes,IDUbicacion
0,1,943
1,2,942
2,3,1004
3,4,952
4,5,861
5,6,904
6,7,763
7,8,916
8,9,834
9,10,402


In [ ]:
##visualizamos municipios por Departamento
df.groupby(["Dpto"]).agg({'Municipio':pd.Series.nunique, 'CANTIDAD':'sum'}).reset_index().sort_values(by="Municipio", ascending=False)

,Dpto,Municipio,CANTIDAD
0,ANTIOQUIA,125,73083.0
4,VALLE DEL CAUCA,42,57572.0
1,CALDAS,27,18862.0
3,RISARALDA,14,12936.0
2,QUINDÍO,12,8075.0


## Eliminar errores ✅
Registros de productos que no son errores de digitación

In [ ]:
df["PRODUCTO"].value_counts()

,count
PRODUCTO,
MANI JOSSEB *210G,136
MANI JOSSEB *440G,81
PAPAS RIZADAS MAYONESA X 31 GM.,47
PQT RIZADA MAYONESA X 45 GR,46
TOSTIEMPANADAS YUPI,45
...,...
MANTEQUILLA LA FINA MESA *125G,1
JABON BARRA ROSAS Y SANDIA *3UND,1
GUANTE DE NITRILO,1


In [ ]:
df = df[df['PRODUCTO'] != "."]
df = df[df['PRODUCTO'] != "ECONOMICA"]

## Eliminar Columnas con demasiados valores nulos ☑️

In [ ]:
##Eliminaremos columnas con demasiados nulos
df = df.dropna(axis=1, thresh=len(df) - df.shape[0]*0.1) ##eliminar columnas con mas del 10% de datos nulos
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13238 entries, 0 to 16382
Data columns (total 60 columns):
 #   Column                                                                                                                           Non-Null Count  Dtype         
---  ------                                                                                                                           --------------  -----         
 0   PRODUCTO                                                                                                                         13238 non-null  object        
 1   FECHA                                                                                                                            13238 non-null  datetime64[ns]
 2   IDUbicacion                                                                                                                      13238 non-null  int64         
 3   CANTIDAD                                                          

## Llenar valores nulos restantes ☑️

In [ ]:
###llenamos los nulos restantes con el promedio de la columna
df = df.apply(lambda col: col.fillna(col.mean()) if col.dtype == 'float64' else col.fillna(col.value_counts().index[0]) if col.dtype == 'object' else col)

# Mostrar el dataframe resultante
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 13238 entries, 0 to 16382
Data columns (total 60 columns):
 #   Column                                                                                                                           Non-Null Count  Dtype         
---  ------                                                                                                                           --------------  -----         
 0   PRODUCTO                                                                                                                         13238 non-null  object        
 1   FECHA                                                                                                                            13238 non-null  datetime64[ns]
 2   IDUbicacion                                                                                                                      13238 non-null  int64         
 3   CANTIDAD                                                          

## Eliminar meses inconsistentes o incompletos ☑️

In [ ]:
df.pivot_table(index="subcategory", columns="mes", values="CANTIDAD", aggfunc="sum")

mes,1,2,3,4,5,6,7,8,9,10,11,12
subcategory,,,,,,,,,,,,
Aseo de hogar,395.0,310.0,319.0,295.0,141.0,184.0,281.0,394.0,327.0,213.0,87.0,190.0
Bebidas,372.0,219.0,759.0,414.0,469.0,271.0,212.0,612.0,569.0,109.0,233.0,441.0
Belleza,244.0,308.0,321.0,322.0,127.0,91.0,225.0,116.0,176.0,18.0,27.0,30.0
Charcutería,450.0,248.0,262.0,231.0,307.0,390.0,275.0,476.0,757.0,211.0,163.0,304.0
Cuidado Personal,1418.0,2053.0,1820.0,1499.0,1048.0,1257.0,426.0,763.0,641.0,63.0,108.0,161.0
Cuidado de ropa y calzado,321.0,932.0,853.0,609.0,167.0,159.0,183.0,166.0,223.0,75.0,24.0,120.0
Despensa,3075.0,2638.0,3895.0,1338.0,1558.0,1297.0,1133.0,1544.0,1614.0,324.0,596.0,924.0
Dulces y postres,2065.0,1688.0,2742.0,2860.0,2026.0,2186.0,1492.0,2099.0,2419.0,1234.0,867.0,1314.0
Frutas y Verduras,3804.0,3327.0,3875.0,3861.0,3676.0,3835.0,2499.0,3863.0,3228.0,1120.0,1101.0,2076.0


In [ ]:
df["mes"].value_counts()


,count
mes,
3,1525
4,1422
1,1384
2,1366
8,1339
6,1308
5,1194
9,1174
7,985


In [ ]:
#eliminar ultimos 3 meses del dataset - DEFINIR SI SE BORRAN O NO!
df = df[df["FECHA"]<="2022-12-31"]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13238 entries, 0 to 16382
Data columns (total 60 columns):
 #   Column                                                                                                                           Non-Null Count  Dtype         
---  ------                                                                                                                           --------------  -----         
 0   PRODUCTO                                                                                                                         13238 non-null  object        
 1   FECHA                                                                                                                            13238 non-null  datetime64[ns]
 2   IDUbicacion                                                                                                                      13238 non-null  int64         
 3   CANTIDAD                                                          

## Agregación de los datos
En este notebook se hace la agregación por subcategoria, departamento y fecha. Se suman las cantidades para estos niveles de agregación, y las variables socioeconomicas son promediadas en cada departamento.

In [ ]:
df.shape

(13238, 60)

In [ ]:
##obtener por separado las variables socioeconimicas promedio por subregion
df_subregiones = df.copy(deep=True)
df_subregiones = df_subregiones.groupby(['Dpto']).agg(
    {'Cuarta medida correctiva al Código Nacional de Policía más impuesta en  la entidad territorial':'mean',
            'Fallecidos por siniestros viales':'mean', 'Porcentaje población de hombres':'mean',
            'Porcentaje población de mujeres':'mean',
            'Primera medida correctiva al Código Nacional de Policía más impuesta en  la entidad territorial':'mean',
            'Proporción en que son impuestas otras medidas correctivas al Código Nacional de Policía más impuesta en  la entidad territorial':'mean',
            'Puntaje SISBEN: Promedio - Cabecera':'mean',
            'Quinta medida correctiva al Código Nacional de Policía más impuesta en  la entidad territorial':'mean',
            'Segunda medida correctiva al Código Nacional de Policía más impuesta en  la entidad territorial':'mean',
            'Tasa de exámenes médico legales por presunto delito sexual contra niños y niñas de 0 a 5 años':'mean',
            'Tasa de extorsión por cada 100.000 habitantes':'mean',
            'Tasa de homicidio intencional en hombres por cada 100.000 habitantes':'mean',
            'Tasa de homicidios en accidentes de tránsito por cada 100.000 habitantes':'mean',
            'Tasa de hurto a personas por cada 100.000 habitantes':'mean',
            'Tasa de lesiones en accidentes de tránsito por cada 100.000 habitantes':'mean',
            'Tasa de secuestro por cada 100.000 habitantes':'mean',
            'Tasa de violencia intrafamiliar por cada 100.000 habitantes':'mean',
            'Tercera medida correctiva al Código Nacional de Policía más impuesta en  la entidad territorial':'mean',
            'Índice de Interrelación de Problemáticas':'mean'})
df_subregiones = df_subregiones.reset_index()
df_subregiones.sample()

,Dpto,Cuarta medida correctiva al Código Nacional de Policía más impuesta en la entidad territorial,Fallecidos por siniestros viales,Porcentaje población de hombres,Porcentaje población de mujeres,Primera medida correctiva al Código Nacional de Policía más impuesta en la entidad territorial,Proporción en que son impuestas otras medidas correctivas al Código Nacional de Policía más impuesta en la entidad territorial,Puntaje SISBEN: Promedio - Cabecera,Quinta medida correctiva al Código Nacional de Policía más impuesta en la entidad territorial,Segunda medida correctiva al Código Nacional de Policía más impuesta en la entidad territorial,Tasa de exámenes médico legales por presunto delito sexual contra niños y niñas de 0 a 5 años,Tasa de extorsión por cada 100.000 habitantes,Tasa de homicidio intencional en hombres por cada 100.000 habitantes,Tasa de homicidios en accidentes de tránsito por cada 100.000 habitantes,Tasa de hurto a personas por cada 100.000 habitantes,Tasa de lesiones en accidentes de tránsito por cada 100.000 habitantes,Tasa de secuestro por cada 100.000 habitantes,Tasa de violencia intrafamiliar por cada 100.000 habitantes,Tercera medida correctiva al Código Nacional de Policía más impuesta en la entidad territorial,Índice de Interrelación de Problemáticas
2,QUINDÍO,1.808067,13.516837,48.947956,51.052044,19.436447,3.051772,39.993045,0.969449,5.838102,227.000979,14.143619,76.996506,18.491102,340.663812,112.446621,0.121083,137.037976,2.898936,22.697936


In [ ]:
df_grouped = df.groupby(['FECHA', 'mes', 'dia_num',
       'subcategory', 'Dia', 'Tipo Dia',
       'Dpto']).agg(
           {'CANTIDAD':'sum'
            }).reset_index()

In [ ]:
# df = df.merge(df_subregiones, on="Dpto")
df_grouped = df_grouped.merge(df_subregiones, on="Dpto")
del df_subregiones

df_grouped.shape ##la agrupacion lleva el dataset a un numero de 5364 filas

(5364, 27)

In [ ]:
df_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5364 entries, 0 to 5363
Data columns (total 27 columns):
 #   Column                                                                                                                           Non-Null Count  Dtype         
---  ------                                                                                                                           --------------  -----         
 0   FECHA                                                                                                                            5364 non-null   datetime64[ns]
 1   mes                                                                                                                              5364 non-null   int32         
 2   dia_num                                                                                                                          5364 non-null   int32         
 3   subcategory                                                    

In [ ]:
def calcular_caracteristicas_ventana(df, ventanas=[1, 3, 7]):
    """
    Calcula características de ventanas móviles para cada combinación de Subregión y subcategoría.

    Parámetros:
    -----------
    df : pandas.DataFrame
        DataFrame con los datos. Debe contener las columnas 'Dpto', 'subcategory',
        'FECHA' y 'CANTIDAD'.
    ventanas : list, opcional
        Lista con los tamaños de ventana (en registros anteriores) para calcular las métricas.
        Por defecto [1, 3, 7].

    Retorna:
    --------
    pandas.DataFrame
        DataFrame original con las columnas adicionales de características de ventana.
    """
    from scipy.stats import linregress, skew, kurtosis
    import numpy as np
    import pandas as pd

    # Hacemos una copia para no modificar el original
    df_resultado = df.copy(deep=True)

    # Aseguramos que el dataframe esté ordenado correctamente
    df_resultado = df_resultado.sort_values(by=['Dpto', 'subcategory', 'FECHA'])

    # Inicializar las columnas de características con NaN
    for ventana in ventanas:
        df_resultado[f'ROLLING_MEAN_{ventana}D'] = np.nan
        df_resultado[f'SLOPE_{ventana}D'] = np.nan
        df_resultado[f'ROLLING_STD_{ventana}D'] = np.nan
        df_resultado[f'ROLLING_MIN_{ventana}D'] = np.nan
        df_resultado[f'ROLLING_MAX_{ventana}D'] = np.nan
        df_resultado[f'ROLLING_SKEW_{ventana}D'] = np.nan
        df_resultado[f'ROLLING_KURTOSIS_{ventana}D'] = np.nan

    # Función para calcular la pendiente en una ventana
    def calcular_slope(valores):
        if len(valores) > 1:
            return linregress(range(len(valores)), valores).slope
        return 0

    # Función para calcular las métricas para una ventana dada
    def calcular_metricas_ventana(valores, ventana):
        # Si no hay suficientes valores para la ventana, devolvemos valores nulos
        if len(valores) == 0:
            return {
                'mean': np.nan,
                'slope': np.nan,
                'std': np.nan,
                'min': np.nan,
                'max': np.nan,
                'skew': np.nan,
                'kurt': np.nan
            }

        # Tomamos solo los últimos 'ventana' valores
        valores_ventana = valores[-ventana:] if len(valores) >= ventana else valores

        # Calculamos las métricas
        return {
            'mean': np.mean(valores_ventana),
            'slope': calcular_slope(valores_ventana),
            'std': np.std(valores_ventana) if len(valores_ventana) > 1 else 0,
            'min': np.min(valores_ventana),
            'max': np.max(valores_ventana),
            'skew': skew(valores_ventana) if len(valores_ventana) > 2 else 0,
            'kurt': kurtosis(valores_ventana) if len(valores_ventana) > 2 else 0
        }

    # Procesamos cada grupo (Subregión, subcategoría) por separado
    grupos = df_resultado.groupby(['Dpto', 'subcategory'])
    df_final = pd.DataFrame()

    # Procesamos cada grupo
    for (subregion, subcategoria), grupo in grupos:
        # Reiniciamos el historial para cada grupo
        historial_cantidades = []

        # Ordenamos por fecha dentro del grupo
        grupo = grupo.sort_values('FECHA').reset_index(drop=True)

        # Para cada fila en el grupo
        for i, row in grupo.iterrows():
            # Para cada ventana, calculamos las métricas basadas en los valores anteriores
            for ventana in ventanas:
                # Solo consideramos datos históricos (no incluimos el registro actual)
                if len(historial_cantidades) > 0:
                    metricas = calcular_metricas_ventana(historial_cantidades, ventana)

                    # Asignamos las métricas a la fila actual
                    grupo.at[i, f'ROLLING_MEAN_{ventana}D'] = metricas['mean']
                    grupo.at[i, f'SLOPE_{ventana}D'] = metricas['slope']
                    grupo.at[i, f'ROLLING_STD_{ventana}D'] = metricas['std']
                    grupo.at[i, f'ROLLING_MIN_{ventana}D'] = metricas['min']
                    grupo.at[i, f'ROLLING_MAX_{ventana}D'] = metricas['max']
                    grupo.at[i, f'ROLLING_SKEW_{ventana}D'] = metricas['skew']
                    grupo.at[i, f'ROLLING_KURTOSIS_{ventana}D'] = metricas['kurt']

            # Después de calcular todas las métricas, agregamos el valor actual al historial
            historial_cantidades.append(row['CANTIDAD'])

        # Agregamos el grupo procesado al DataFrame de resultados
        df_final = pd.concat([df_final, grupo])

    # Reseteamos el índice para que quede en formato estándar
    df_final = df_final.reset_index(drop=True)

    return df_final

def rellenar_dias_faltantes_con_cero(df):
    """
    Rellena con ceros los datos para los días faltantes en el DataFrame.

    Para cada combinación de subcategoría y subregión, identifica saltos en las fechas
    y crea registros con valor cero para los días faltantes.

    Parámetros:
    -----------
    df : pandas.DataFrame
        DataFrame con los datos. Debe contener las columnas 'Dpto', 'subcategory',
        'FECHA' y 'CANTIDAD'.

    Retorna:
    --------
    pandas.DataFrame
        DataFrame con los días faltantes rellenados con ceros.
    """
    import pandas as pd
    import numpy as np
    from datetime import timedelta

    # Crear una copia del DataFrame original para no modificarlo
    df_resultado = df.copy()

    # Aseguramos que la fecha sea de tipo datetime
    df_resultado['FECHA'] = pd.to_datetime(df_resultado['FECHA'])

    # Lista para almacenar DataFrames de cada grupo con datos rellenados
    dfs_rellenados = []

    # Agrupar por subcategoría y subregión
    grupos = df_resultado.groupby(['Dpto', 'subcategory'])

    # Recorrer cada grupo
    for (subregion, subcategoria), grupo in grupos:
        # Ordenar por fecha dentro del grupo y eliminar posibles duplicados de fecha
        grupo = grupo.sort_values('FECHA')

        # Eliminar duplicados de fecha si existen, conservando el primer registro
        grupo = grupo.drop_duplicates(subset=['FECHA'], keep='first').reset_index(drop=True)

        # Lista para almacenar filas originales y nuevas filas con ceros
        filas_totales = []
        fechas_existentes = set(grupo['FECHA'].dt.date)

        # Recorrer filas del grupo para detectar días faltantes
        for i in range(len(grupo) - 1):
            # Añadir la fila actual
            filas_totales.append(grupo.iloc[i].copy())

            # Calcular la diferencia de días entre fechas consecutivas
            fecha_actual = grupo.iloc[i]['FECHA']
            fecha_siguiente = grupo.iloc[i + 1]['FECHA']
            dias_diferencia = (fecha_siguiente - fecha_actual).days

            # Si hay más de 1 día de diferencia, rellenar con ceros
            if dias_diferencia > 1:
                # Crear fechas intermedias
                for j in range(1, dias_diferencia):
                    fecha_rellenar = fecha_actual + timedelta(days=j)

                    # Verificar que esta fecha no exista ya en el conjunto original
                    if fecha_rellenar.date() not in fechas_existentes:
                        # Crear fila con ceros
                        fila_cero = grupo.iloc[i].copy()
                        fila_cero['FECHA'] = fecha_rellenar

                        # Asignar cero a CANTIDAD
                        fila_cero['CANTIDAD'] = 0

                        # Para otras columnas numéricas, mantener el valor anterior
                        # Otra opción sería asignarles también ceros o valores nulos

                        # Añadir la fila con cero
                        filas_totales.append(fila_cero)

        # Añadir la última fila del grupo
        if len(grupo) > 0:
            filas_totales.append(grupo.iloc[-1].copy())

        # Convertir la lista de filas a DataFrame
        df_grupo_rellenado = pd.DataFrame(filas_totales)

        # Añadir al resultado
        dfs_rellenados.append(df_grupo_rellenado)

    # Concatenar todos los grupos
    df_rellenado = pd.concat(dfs_rellenados, ignore_index=True)

    # Verificar si hay duplicados después del relleno
    duplicados = df_rellenado.duplicated(subset=['Dpto', 'subcategory', 'FECHA'], keep=False)
    if duplicados.any():
        print(f"¡Advertencia! Se encontraron {duplicados.sum()} registros duplicados después del relleno.")
        # Eliminar duplicados conservando el primer registro
        df_rellenado = df_rellenado.drop_duplicates(subset=['Dpto', 'subcategory', 'FECHA'], keep='first')

    # Ordenar el DataFrame final
    df_rellenado = df_rellenado.sort_values(by=['Dpto', 'subcategory', 'FECHA']).reset_index(drop=True)

    # Comparación de tamaños
    n_original = len(df)
    n_rellenado = len(df_rellenado)
    print(f"Tamaño original: {n_original} registros")
    print(f"Tamaño después de rellenar: {n_rellenado} registros")
    print(f"Registros añadidos: {n_rellenado - n_original} registros")

    return df_rellenado

def interpolar_dias_faltantes(df):
    """
    Interpola los datos para los días faltantes en el DataFrame,
    asegurándose de que no se creen registros duplicados para la misma fecha.

    Para cada combinación de subcategoría y subregión, identifica saltos en las fechas
    y crea registros interpolados para los días faltantes.

    Parámetros:
    -----------
    df : pandas.DataFrame
        DataFrame con los datos. Debe contener las columnas 'Dpto', 'subcategory',
        'FECHA' y 'CANTIDAD'.

    Retorna:
    --------
    pandas.DataFrame
        DataFrame con los días faltantes interpolados.
    """
    import pandas as pd
    import numpy as np
    from datetime import timedelta

    # Crear una copia del DataFrame original para no modificarlo
    df_resultado = df.copy(deep=True)

    # Aseguramos que la fecha sea de tipo datetime
    df_resultado['FECHA'] = pd.to_datetime(df_resultado['FECHA'])

    # Lista para almacenar DataFrames de cada grupo con datos interpolados
    dfs_interpolados = []

    # Agrupar por subcategoría y subregión
    grupos = df_resultado.groupby(['Dpto', 'subcategory'])

    # Recorrer cada grupo
    for (subregion, subcategoria), grupo in grupos:
        # Ordenar por fecha dentro del grupo y eliminar posibles duplicados de fecha
        grupo = grupo.sort_values('FECHA')

        # Eliminar duplicados de fecha si existen, conservando el primer registro
        grupo = grupo.drop_duplicates(subset=['FECHA'], keep='first').reset_index(drop=True)

        # Lista para almacenar filas originales e interpoladas
        filas_totales = []
        fechas_existentes = set(grupo['FECHA'].dt.date)

        # Recorrer filas del grupo para detectar días faltantes
        for i in range(len(grupo) - 1):
            # Añadir la fila actual
            filas_totales.append(grupo.iloc[i].copy())

            # Calcular la diferencia de días entre fechas consecutivas
            fecha_actual = grupo.iloc[i]['FECHA']
            fecha_siguiente = grupo.iloc[i + 1]['FECHA']
            dias_diferencia = (fecha_siguiente - fecha_actual).days

            # Si hay más de 1 día de diferencia, interpolar
            if dias_diferencia > 1:
                # Crear fechas intermedias
                for j in range(1, dias_diferencia):
                    fecha_interpolar = fecha_actual + timedelta(days=j)

                    # Verificar que esta fecha no exista ya en el conjunto original
                    if fecha_interpolar.date() not in fechas_existentes:
                        # Crear fila interpolada
                        fila_interpolada = grupo.iloc[i].copy()
                        fila_interpolada['FECHA'] = fecha_interpolar

                        # Interpolar CANTIDAD linealmente
                        cantidad_actual = grupo.iloc[i]['CANTIDAD']
                        cantidad_siguiente = grupo.iloc[i + 1]['CANTIDAD']
                        fila_interpolada['CANTIDAD'] = cantidad_actual + (cantidad_siguiente - cantidad_actual) * j / dias_diferencia

                        # Para otras columnas numéricas, también se podría interpolar si es necesario
                        for col in grupo.select_dtypes(include=['float64', 'int64']).columns:
                            if col != 'CANTIDAD' and col in grupo.columns:
                                valor_actual = grupo.iloc[i][col]
                                valor_siguiente = grupo.iloc[i + 1][col]
                                # Verificar si los valores no son NaN
                                if pd.notna(valor_actual) and pd.notna(valor_siguiente):
                                    fila_interpolada[col] = valor_actual + (valor_siguiente - valor_actual) * j / dias_diferencia

                        # Añadir la fila interpolada
                        filas_totales.append(fila_interpolada)

        # Añadir la última fila del grupo
        if len(grupo) > 0:
            filas_totales.append(grupo.iloc[-1].copy())

        # Convertir la lista de filas a DataFrame
        df_grupo_interpolado = pd.DataFrame(filas_totales)

        # Añadir al resultado
        dfs_interpolados.append(df_grupo_interpolado)

    # Concatenar todos los grupos
    df_interpolado = pd.concat(dfs_interpolados, ignore_index=True)

    # Verificar si hay duplicados después de la interpolación
    duplicados = df_interpolado.duplicated(subset=['Dpto', 'subcategory', 'FECHA'], keep=False)
    if duplicados.any():
        print(f"¡Advertencia! Se encontraron {duplicados.sum()} registros duplicados después de la interpolación.")
        # Eliminar duplicados conservando el primer registro
        df_interpolado = df_interpolado.drop_duplicates(subset=['Dpto', 'subcategory', 'FECHA'], keep='first')

    # Ordenar el DataFrame final
    df_interpolado = df_interpolado.sort_values(by=['Dpto', 'subcategory', 'FECHA']).reset_index(drop=True)

    # Comparación de tamaños
    n_original = len(df)
    n_interpolado = len(df_interpolado)
    print(f"Tamaño original: {n_original} registros")
    print(f"Tamaño después de interpolar: {n_interpolado} registros")
    print(f"Registros añadidos: {n_interpolado - n_original} registros")

    return df_interpolado

In [ ]:
df_completo_rellenado = rellenar_dias_faltantes_con_cero(df_grouped)
df_completo_interpolado = interpolar_dias_faltantes(df_grouped)

Tamaño original: 5364 registros
Tamaño después de rellenar: 27312 registros
Registros añadidos: 21948 registros
Tamaño original: 5364 registros
Tamaño después de interpolar: 27312 registros
Registros añadidos: 21948 registros


In [ ]:
df_features_rellenado = calcular_caracteristicas_ventana(df_completo_rellenado, ventanas=[1, 3, 7])
df_features_interpolado = calcular_caracteristicas_ventana(df_completo_interpolado, ventanas=[1, 3, 7])
df_features_rellenado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27312 entries, 0 to 27311
Data columns (total 48 columns):
 #   Column                                                                                                                           Non-Null Count  Dtype         
---  ------                                                                                                                           --------------  -----         
 0   FECHA                                                                                                                            27312 non-null  datetime64[ns]
 1   mes                                                                                                                              27312 non-null  int32         
 2   dia_num                                                                                                                          27312 non-null  int32         
 3   subcategory                                                  

In [ ]:
df_features_interpolado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27312 entries, 0 to 27311
Data columns (total 48 columns):
 #   Column                                                                                                                           Non-Null Count  Dtype         
---  ------                                                                                                                           --------------  -----         
 0   FECHA                                                                                                                            27312 non-null  datetime64[ns]
 1   mes                                                                                                                              27312 non-null  int32         
 2   dia_num                                                                                                                          27312 non-null  int32         
 3   subcategory                                                  

In [ ]:
df_features_rellenado.loc[(df_features_rellenado['Dpto']=='QUINDÍO') &  (df_features_rellenado['subcategory']=='Panadería y Pastelería'),
       ['FECHA', 'subcategory', 'Dpto', 'CANTIDAD', 'ROLLING_MEAN_1D', 'ROLLING_MEAN_3D','ROLLING_MEAN_7D',
        'ROLLING_STD_3D', 'ROLLING_STD_7D']]
##rolling mean de los x registros anteriores de ventas (no dias calendario). aclarar en articulo

,FECHA,subcategory,Dpto,CANTIDAD,ROLLING_MEAN_1D,ROLLING_MEAN_3D,ROLLING_MEAN_7D,SLOPE_1D,ROLLING_STD_3D,ROLLING_STD_7D
14543,2022-01-05,Panadería y Pastelería,QUINDÍO,6.0,NaN,NaN,NaN,NaN,NaN,NaN
14544,2022-01-06,Panadería y Pastelería,QUINDÍO,0.0,6.0,6.0,6.0,0.0,0.000000,0.000000
14545,2022-01-07,Panadería y Pastelería,QUINDÍO,0.0,0.0,3.0,3.0,0.0,3.000000,3.000000
14546,2022-01-08,Panadería y Pastelería,QUINDÍO,0.0,0.0,2.0,2.0,0.0,2.828427,2.828427
14547,2022-01-09,Panadería y Pastelería,QUINDÍO,0.0,0.0,0.0,1.5,0.0,0.000000,2.598076
...,...,...,...,...,...,...,...,...,...,...
14841,2022-10-30,Panadería y Pastelería,QUINDÍO,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
14842,2022-10-31,Panadería y Pastelería,QUINDÍO,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
14843,2022-11-01,Panadería y Pastelería,QUINDÍO,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
14844,2022-11-02,Panadería y Pastelería,QUINDÍO,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000


In [ ]:
df_features_interpolado.loc[(df_features_interpolado['Dpto']=='QUINDÍO') &  (df_features_interpolado['subcategory']=='Panadería y Pastelería'),
       ['FECHA', 'subcategory', 'Dpto', 'CANTIDAD', 'ROLLING_MEAN_1D', 'ROLLING_MEAN_3D','ROLLING_MEAN_7D',
        'ROLLING_STD_3D', 'ROLLING_STD_7D']]
##rolling mean de los x registros anteriores de ventas (no dias calendario). aclarar en articulo

,FECHA,subcategory,Dpto,CANTIDAD,ROLLING_MEAN_1D,ROLLING_MEAN_3D,ROLLING_MEAN_7D,ROLLING_STD_3D,ROLLING_STD_7D
14543,2022-01-05,Panadería y Pastelería,QUINDÍO,6.0000,NaN,NaN,NaN,NaN,NaN
14544,2022-01-06,Panadería y Pastelería,QUINDÍO,6.0000,6.0000,6.0000,6.0000,0.000000,0.000
14545,2022-01-07,Panadería y Pastelería,QUINDÍO,6.0000,6.0000,6.0000,6.0000,0.000000,0.000
14546,2022-01-08,Panadería y Pastelería,QUINDÍO,6.0000,6.0000,6.0000,6.0000,0.000000,0.000
14547,2022-01-09,Panadería y Pastelería,QUINDÍO,6.0000,6.0000,6.0000,6.0000,0.000000,0.000
...,...,...,...,...,...,...,...,...,...
14841,2022-10-30,Panadería y Pastelería,QUINDÍO,8.7500,8.4375,8.1250,7.5000,0.255155,0.625
14842,2022-10-31,Panadería y Pastelería,QUINDÍO,9.0625,8.7500,8.4375,7.8125,0.255155,0.625
14843,2022-11-01,Panadería y Pastelería,QUINDÍO,9.3750,9.0625,8.7500,8.1250,0.255155,0.625
14844,2022-11-02,Panadería y Pastelería,QUINDÍO,9.6875,9.3750,9.0625,8.4375,0.255155,0.625


## Valores atipicos ✅

In [ ]:
# Enfoque para tratar valores atípicos sin eliminarlos
def limitar_valores_atipicos(df, multiplicador_iqr=5):
    """
    Limita los valores atípicos superiores al máximo normal para cada combinación de
    subcategoría y subregión, en lugar de eliminarlos.

    Parámetros:
    -----------
    df : pandas.DataFrame
        DataFrame con los datos. Debe contener las columnas 'Dpto', 'subcategory',
        y 'CANTIDAD'.
    multiplicador_iqr : float, opcional
        Factor para multiplicar el IQR y determinar valores atípicos.
        Por defecto es 5.

    Retorna:
    --------
    pandas.DataFrame
        DataFrame con los valores atípicos ajustados.
    """
    # Crear una copia para no modificar el original
    df_resultado = df.copy()

    # Contar registros originales
    n_original = len(df_resultado)
    n_modificados = 0

    # Procesar cada combinación de subregión y subcategoría
    grupos = df_resultado.groupby(['Dpto', 'subcategory'])

    for (subregion, subcategoria), grupo in grupos:
        # Calcular cuartiles e IQR para esta combinación específica
        Q1 = np.percentile(grupo['CANTIDAD'], 15, interpolation='midpoint')
        Q3 = np.percentile(grupo['CANTIDAD'], 85, interpolation='midpoint')
        IQR = Q3 - Q1

        # Calcular límite superior para valores atípicos
        limite_superior = Q3 + multiplicador_iqr * IQR

        # Identificar registros con valores atípicos
        indices_atipicos = grupo[grupo['CANTIDAD'] > limite_superior].index
        n_modificados += len(indices_atipicos)

        if len(indices_atipicos) > 0:
            # Ajustar valores atípicos al límite superior
            df_resultado.loc[indices_atipicos, 'CANTIDAD'] = limite_superior

            print(f"Ajustados {len(indices_atipicos)} valores atípicos para {subcategoria} en {subregion}. "
                  f"Límite superior: {limite_superior:.2f}")

    # Mostrar resumen
    print(f"\nResumen de ajustes:")
    print(f"Total de registros: {n_original}")
    print(f"Registros modificados: {n_modificados} ({(n_modificados/n_original)*100:.2f}%)")

    return df_resultado

In [ ]:
# En lugar de eliminar valores atípicos, los limitamos al máximo normal por subregión y subcategoría
df_features_rellenado = limitar_valores_atipicos(df_features_rellenado, multiplicador_iqr=3)
df_features_interpolado = limitar_valores_atipicos(df_features_interpolado, multiplicador_iqr=3)

# Ya no es necesario reiniciar el índice, pues no eliminamos filas
# df.reset_index(drop=True, inplace=True)

print(f"Forma del DataFrame después del ajuste: {df_features_rellenado.shape}")

Ajustados 12 valores atípicos para Aseo de hogar en ANTIOQUIA. Límite superior: 24.00
Ajustados 4 valores atípicos para Bebidas en ANTIOQUIA. Límite superior: 48.00
Ajustados 14 valores atípicos para Belleza en ANTIOQUIA. Límite superior: 12.00
Ajustados 4 valores atípicos para Charcutería en ANTIOQUIA. Límite superior: 48.00
Ajustados 11 valores atípicos para Cuidado Personal en ANTIOQUIA. Límite superior: 104.00
Ajustados 30 valores atípicos para Cuidado de ropa y calzado en ANTIOQUIA. Límite superior: 20.00
Ajustados 7 valores atípicos para Despensa en ANTIOQUIA. Límite superior: 168.00
Ajustados 16 valores atípicos para Dulces y postres en ANTIOQUIA. Límite superior: 144.00
Ajustados 2 valores atípicos para Frutas y Verduras en ANTIOQUIA. Límite superior: 392.00
Ajustados 18 valores atípicos para Limpieza de cocina en ANTIOQUIA. Límite superior: 72.00
Ajustados 5 valores atípicos para Lácteos, huevos y refrigerados en ANTIOQUIA. Límite superior: 120.00
Ajustados 49 valores atípicos

In [ ]:
fig = px.box(df_features_rellenado, y='CANTIDAD', hover_data=["Dpto", "subcategory"],
             title='Distribución de la Subregion vs. Cantidades',
             color_discrete_sequence=px.colors.qualitative.Prism_r)
fig.show()

del fig

In [ ]:
fig = px.box(df_features_interpolado, y='CANTIDAD', hover_data=["Dpto", "subcategory"],
             title='Distribución de la Subregion vs. Cantidades',
             color_discrete_sequence=px.colors.qualitative.Prism_r)
fig.show()

del fig

In [ ]:
# Agrupar los datos por fecha y subregión, calculando el promedio de CANTIDAD entre todas las categorías
df_avg_by_region = df_features_rellenado.groupby(['FECHA', 'Dpto'])['CANTIDAD'].mean().reset_index()

# Crear el gráfico con una línea diferente para cada subregión
fig = px.line(
    df_avg_by_region,
    x='FECHA',
    y='CANTIDAD',
    color='Dpto',
    title='Promedio de Cantidades Vendidas por Subregión (Todas las Categorías)',
    labels={
        'FECHA': 'Fecha',
        'CANTIDAD': 'Cantidad Promedio',
        'Dpto': 'Subregión'
    },
    color_discrete_sequence=px.colors.qualitative.Bold
)

# Personalizar el diseño
fig.update_layout(
    template='plotly_white',
    legend_title_text='Subregión',
    xaxis_title='Fecha',
    yaxis_title='Cantidad Promedio por Categoría',
    hovermode='x unified',
    legend={'itemsizing': 'constant'}
)

# Mejorar la legibilidad si hay muchas líneas
fig.update_traces(line=dict(width=1.5))

# Mostrar el gráfico
fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display

# Crear widgets para selección de subregión y categoría
subregion_dropdown = widgets.Dropdown(
    options=sorted(df_features_rellenado['Dpto'].unique()),
    description='Subregión:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

category_dropdown = widgets.Dropdown(
    options=sorted(df_features_rellenado['subcategory'].unique()),
    description='Categoría:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# Función para actualizar el gráfico basado en las selecciones
def update_plot(subregion, category):
    # Filtrar datos según selecciones
    filtered_data = df_features_rellenado[
        (df_features_rellenado['Dpto'] == subregion) &
        (df_features_rellenado['subcategory'] == category)
    ].sort_values('FECHA')

    if filtered_data.empty:
        return go.Figure().update_layout(title=f"No hay datos para {category} en {subregion}")

    # Crear figura con dos ejes Y
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Añadir línea para CANTIDAD
    fig.add_trace(
        go.Scatter(
            x=filtered_data['FECHA'],
            y=filtered_data['CANTIDAD'],
            name='Cantidad',
            line=dict(color='blue', width=3),
            mode='lines+markers'
        ),
        secondary_y=False
    )

    # Añadir líneas para los promedios móviles (rolling means)
    for window, color in zip([1, 3, 7], ['red', 'green', 'purple']):
        column_name = f'ROLLING_MEAN_{window}W'
        if column_name in filtered_data.columns:
            fig.add_trace(
                go.Scatter(
                    x=filtered_data['FECHA'],
                    y=filtered_data[column_name],
                    name=f'Media Móvil ({window})',
                    line=dict(color=color, width=2, dash='dot'),
                    mode='lines'
                ),
                secondary_y=False
            )

    # Configurar títulos y leyendas
    fig.update_layout(
        title=f"{category} en {subregion}: Cantidad y Promedios Móviles",
        template='plotly_white',
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='center', x=0.5),
        height=600,
        hovermode='x unified'
    )

    fig.update_xaxes(title_text='Fecha', tickangle=45)
    fig.update_yaxes(title_text='Cantidad', secondary_y=False)

    return fig

# Función para manejar los cambios en los widgets
def on_change(change):
    with output:
        output.clear_output(wait=True)
        fig = update_plot(subregion_dropdown.value, category_dropdown.value)
        fig.show()

# Configurar los widgets para actualizar el gráfico cuando cambian
subregion_dropdown.observe(on_change, names='value')
category_dropdown.observe(on_change, names='value')

# Crear el área de salida
output = widgets.Output()

# Mostrar los widgets y el área de salida
display(widgets.VBox([
    widgets.HBox([subregion_dropdown, category_dropdown]),
    output
]))

# Mostrar el gráfico inicial
with output:
    fig = update_plot(subregion_dropdown.value, category_dropdown.value)
    fig.show()

In [ ]:
df_features_rellenado.to_csv("datosPorDiaDepartamento_ceros.csv", index=False)
df_features_interpolado.to_csv("datosPorDiaDepartamento_interpolado.csv", index=False)

# Referencias

- *scikit-learn:*   
    - https://scikit-learn.org/stable/

- *Gráficas con  plotly:*
    - https://plotly.com/python/

- *Compras Dataset:*
    - https://www.kaggle.com/datasets/davidms8/shopping-map
    
- *Lista Productos Con categoria:*
    - https://www.kaggle.com/datasets/camiloemartinez/productos-consumo-masivo

